# Mapeando el Universo



## Introducción al aprendizaje automático

  * Tomar el dataset "limpio" obtenido del práctico anterior y considerar algunos de los siguientes conjuntos de variables
  
   + modelMag_*, colores_modelMag, devRad_*
   + modelMag_*, colores_modelMag, expRad_*
   + petroMag_*, colores_petroMag, petroRad_*, petroR50_*, petroR90_*
  
  * Emplear el modelo de regresión lineal para estimar la variables _redshift_.
   + Considere utilizar o no regulación
    
  * Utilizar algún método para evaluar la complejidad del/de los modelo/s.
  
  * Evaluar que sucede si se distrimina por tipo de galaxia.
  
  * Emplear algunos (más de uno) de los modelos de clasificación (perceptrón, regresión logística, vecinos más cercanos) para clasificar las galaxias en elípticas y espirales a partir de las variables elegidas.
  
   + Computar las metricas accuracy,precision, recall, F1, matriz de confusión
 
   + Visualización de la frontera de decisión
   

### Lectura de datos

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
import zipfile

path_to_zip_file = '../data/DiploDatos_clean.zip'
directory_to_extract_to = '../data/'

with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
    zip_ref.extractall(directory_to_extract_to)

In [4]:
filename = '../data/DiploDatos_clean.csv'

In [5]:
df = pd.read_csv(filename,index_col=0)
df

,z,subClass_AGN,subClass_AGN BROADLINE,subClass_BROADLINE,subClass_STARBURST,subClass_STARBURST BROADLINE,subClass_STARFORMING,subClass_STARFORMING BROADLINE,subClass_UNKNOWN,velDisp,...,extinction_i,extinction_z,modelColor_ug,modelColor_gr,modelColor_ri,modelColor_iz,petroColor_ug,petroColor_gr,petroColor_ri,petroColor_iz
specObjID,,,,,,,,,,,,,,,,,,,,,
957075158303008768,0.012095,0,0,0,1,0,0,0,0,60.39996,...,0.051128,0.038030,0.76524,0.25431,0.09500,0.11706,0.79000,0.35837,0.15143,0.10247
957066087332079616,0.141718,0,0,0,0,0,0,0,1,217.80020,...,0.051117,0.038022,1.95531,1.07714,0.45090,0.35530,2.11901,1.03352,0.43049,0.27852
957065537576265728,0.140137,0,0,0,0,0,0,0,1,138.90830,...,0.051345,0.038191,1.36642,0.85273,0.41891,0.31316,1.07275,0.76730,0.36148,0.22301
957066911965800448,0.066672,0,0,0,0,0,0,0,1,241.28300,...,0.053670,0.039920,1.98755,0.92406,0.42804,0.32709,1.90482,0.94006,0.41266,0.26840
693629150375733248,0.229718,0,0,0,0,0,0,0,1,198.84190,...,0.026088,0.019405,1.96747,1.35756,0.48971,0.40055,2.99483,1.33661,0.49060,0.37582
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
957077632204171264,0.150921,0,0,0,0,0,0,0,1,181.48450,...,0.046565,0.034636,1.97425,1.03221,0.44277,0.30476,1.22845,0.93079,0.40980,0.05350
957077357326264320,0.084226,0,0,0,0,0,1,0,0,43.81417,...,0.052704,0.039202,1.51278,0.67597,0.38023,0.24776,1.51657,0.61436,0.38013,0.17309
957066637087893504,0.141610,0,0,0,0,0,0,0,1,194.91520,...,0.055708,0.041437,2.15777,1.09985,0.45063,0.33057,2.28958,0.96166,0.41758,0.25075
